In [1]:
%load_ext autoreload
%autoreload 2

%load_ext memory_profiler

from vflow import Vset
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from functools import partial

from numpy.testing import assert_equal

In [2]:
def make_train_test():
    X, y = make_classification(n_samples=1000, n_features=5)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return(X_train, X_test, y_train, y_test)

train_test_vset = Vset(name = 'train_test', modules=[make_train_test for i in range(10)])
train_test_vset_lazy = Vset(name = 'train_test', modules=[make_train_test for i in range(10)], lazy=True)

print(list(train_test_vset().values())[0][0])

[[-0.65947694 -1.44201367 -1.24972018 -0.30548676  0.82519115]
 [ 0.97727562 -0.52509499  2.0965408  -0.3069418  -1.85819849]
 [-0.42756623  0.40716562 -0.40488646 -1.45703011 -0.51797723]
 ...
 [ 0.16119377  1.12721429 -0.4606357   2.45403713  1.78836691]
 [ 0.04460337  0.30707785  0.0388526   0.16250888  0.0628276 ]
 [-0.31037909  1.56655538 -0.46607321 -0.52299742  0.07119645]]


In [3]:
# lazy = True returns a promise
promise = list(train_test_vset_lazy().values())[0]
print(promise)

Unfulfilled VfuncPromise(func=<vflow.vfunc.Vfunc object at 0x7f7d8f045730>, args=())


In [4]:
# calling the promise fulfills it
promise()
print(promise)

Fulfilled VfuncPromise((array([[-7.89904844e-01, -4.91522637e-01,  5.11713998e-01,
        -8.33472733e-01,  6.76109819e-01],
       [-7.54443112e-01, -4.69796512e-01,  4.58537623e-01,
        -7.83833046e-01, -9.38132489e-01],
       [ 2.09091071e+00,  1.30076056e+00, -1.38295620e+00,
         2.21774028e+00, -8.22049103e-01],
       ...,
       [ 6.53793206e-01,  4.19286390e-01,  6.82902482e-01,
         2.42127839e-01,  4.92449339e-01],
       [ 1.12201833e+00,  7.31115685e-01,  2.19763025e+00,
         4.96077261e-04, -3.19369920e-01],
       [ 8.67180335e-01,  5.78354096e-01,  2.87888923e+00,
        -4.77268248e-01, -3.26003689e-01]]), array([[-1.24361296, -0.76569416,  1.52942707, -1.60509138, -0.37407985],
       [-1.50942778, -0.97429061, -2.13378353, -0.33355302, -1.7376628 ],
       [-0.0307683 ,  0.00562255,  2.21934831, -0.9224681 , -0.28334538],
       ...,
       [-1.22546014, -0.75461242,  1.49866829, -1.57824927, -0.61861889],
       [ 2.41062974,  1.49393774, -2.10242

In [5]:
# at this point the value is cached and calling it just returns the cached value
print(promise.called)
print(id(promise.value) == id(promise()))

True
True
